In [17]:
import json
import pandas as pd
from pulp import *

# Manual blacklist - add player names here to exclude them
BLACKLIST = [
    # "FirstName LastName"
    "Oscar Tshiebwe",
    # "Nikola Jokić",
    "Mark Williams"
]

with open('players.json','r', encoding='utf-8') as f:
    data=json.load(f)

# Build candidate list with positions
candidates = []
players = data.get('players', {}).get('result', [])
for p in players:
    player_name = f"{p.get('firstName','')} {p.get('lastName','')}"
    
    # Skip injured players and blacklisted players
    if p.get('status') == "INJ":
        continue
    if player_name in BLACKLIST:
        continue
    
    # Use average of last 2 games as projected points
    hist = p.get('fantasyPointsHistory', [])
    if len(hist) >= 2:
        projected = sum(hist[:2]) / 2
    else:
        continue
    
    salary = p.get('salary', 0)
    positions = p.get('eligiblePositions', [])
    if salary > 0 and positions and projected > 0:
        candidates.append({
            "name": player_name,
            "projectedPoints": projected,
            "salary": salary,
            "positions": positions,
            "id": len(candidates)
        })

# Position requirements
position_slots = {
    'PG': 1,
    'SG': 1, 
    'G': 1,   # PG or SG
    'SF': 1,
    'PF': 1,
    'F': 1,   # SF or PF
    'C': 1,
    'Util': 1  # any position
}
max_salary = 200

# Create optimization problem
prob = LpProblem("DFS_Lineup_Optimizer", LpMaximize)

# Decision variables: 1 if player is selected, 0 otherwise
player_vars = {i: LpVariable(f"player_{i}", cat='Binary') for i in range(len(candidates))}

# Objective: maximize projected points
prob += lpSum([candidates[i]['projectedPoints'] * player_vars[i] for i in range(len(candidates))])

# Constraint: total salary <= 200
prob += lpSum([candidates[i]['salary'] * player_vars[i] for i in range(len(candidates))]) <= max_salary

# Constraint: exactly 8 players selected
prob += lpSum([player_vars[i] for i in range(len(candidates))]) == 8

# Position constraints
# PG slot
pg_players = [i for i, p in enumerate(candidates) if 'PG' in p['positions']]
prob += lpSum([player_vars[i] for i in pg_players]) >= 1

# SG slot
sg_players = [i for i, p in enumerate(candidates) if 'SG' in p['positions']]
prob += lpSum([player_vars[i] for i in sg_players]) >= 1

# G slot (PG or SG)
g_players = [i for i, p in enumerate(candidates) if 'PG' in p['positions'] or 'SG' in p['positions']]
prob += lpSum([player_vars[i] for i in g_players]) >= 3  # PG + SG + G

# SF slot
sf_players = [i for i, p in enumerate(candidates) if 'SF' in p['positions']]
prob += lpSum([player_vars[i] for i in sf_players]) >= 1

# PF slot
pf_players = [i for i, p in enumerate(candidates) if 'PF' in p['positions']]
prob += lpSum([player_vars[i] for i in pf_players]) >= 1

# F slot (SF or PF)
f_players = [i for i, p in enumerate(candidates) if 'SF' in p['positions'] or 'PF' in p['positions']]
prob += lpSum([player_vars[i] for i in f_players]) >= 3  # SF + PF + F

# C slot
c_players = [i for i, p in enumerate(candidates) if 'C' in p['positions']]
prob += lpSum([player_vars[i] for i in c_players]) >= 1

# Solve
prob.solve(PULP_CBC_CMD(msg=0))

# Extract results
if prob.status == 1:  # Optimal solution found
    selected_indices = [i for i in range(len(candidates)) if player_vars[i].varValue == 1]
    selected_players = [candidates[i] for i in selected_indices]
    
    total_salary = sum(p['salary'] for p in selected_players)
    total_points = sum(p['projectedPoints'] for p in selected_players)
    
    results = []
    for player in selected_players:
        results.append({
            'Name': player['name'],
            'Salary': player['salary'],
            'ProjectedPoints': player['projectedPoints'],
            'EligiblePositions': ', '.join(player['positions'])
        })
    
    df = pd.DataFrame(results).sort_values('ProjectedPoints', ascending=False)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    print(f"Total Salary: {total_salary} | Total Projected Points (2-game avg): {total_points:.2f}")
    display(df)
else:
    print(f"No optimal solution found. Status: {LpStatus[prob.status]}")

Total Salary: 198 | Total Projected Points (2-game avg): 355.15


,Name,Salary,ProjectedPoints,EligiblePositions
3,Nikola Jokić,60,82.55,C
1,Cooper Flagg,34,52.50,SF
0,DeMar DeRozan,30,52.45,PF
6,Tyler Kolek,10,43.15,PG
2,Anthony Black,26,41.50,PG
5,Sam Merrill,17,32.60,SG
7,Bones Hyland,11,27.45,PG
4,Spencer Jones,10,22.95,SF


In [18]:
# Find players with maximum positive difference (projected points - salary)
value_players = []
for p in candidates:
    diff = p['projectedPoints'] - p['salary']
    if diff > 0:
        value_players.append({
            'Name': p['name'],
            'Salary': p['salary'],
            'ProjectedPoints': p['projectedPoints'],
            'Difference': diff,
            'EligiblePositions': ', '.join(p['positions'])
        })

value_df = pd.DataFrame(value_players).sort_values('Difference', ascending=False).head(20)
print("\nTop 20 Players by Value (ProjectedPoints - Salary):")
display(value_df)


Top 20 Players by Value (ProjectedPoints - Salary):


,Name,Salary,ProjectedPoints,Difference,EligiblePositions
63,Tyler Kolek,10,43.15,33.15,PG
21,Nikola Jokić,60,82.55,22.55,C
1,DeMar DeRozan,30,52.45,22.45,PF
13,Cooper Flagg,34,52.50,18.50,SF
79,Bones Hyland,11,27.45,16.45,PG
14,Tyus Jones,10,25.80,15.80,PG
51,Sam Merrill,17,32.60,15.60,SG
18,Anthony Black,26,41.50,15.50,PG
22,Jamal Murray,41,55.90,14.90,PG
78,Anthony Edwards,44,58.05,14.05,SG
